In [ ]:
!pip install transformers datasets sentencepiece --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
import pandas as pd
import torch

In [ ]:
data = [
    {"prompt": "Build a website for a bakery", "instruction": "Create a responsive layout with hero image, menu section, contact form, and footer"},
    {"prompt": "Design a portfolio site for a designer", "instruction": "Include hero section, project gallery, and contact form"},
    {"prompt": "Create a landing page for a mobile app", "instruction": "Add app features, download buttons, testimonials, and footer"},
    {"prompt": "Build a blog for travel stories", "instruction": "Include recent posts, categories, author bio, and comment section"},
    {"prompt": "Make a website for a local gym", "instruction": "Add class schedule, trainer profiles, membership plans, and contact form"}
]

df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

In [ ]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess(example):
    model_inputs = tokenizer(example["prompt"], max_length=64, truncation=True, padding="max_length")
    labels = tokenizer(example["instruction"], max_length=64, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess, remove_columns=["prompt", "instruction"])

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_dir="./logs",
    save_strategy="no",
    predict_with_generate=True,
    report_to="none"  # disables wandb
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/tmp/ipython-input-1741520534.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss


TrainOutput(global_step=30, training_loss=9.960376993815105, metrics={'train_runtime': 10.3788, 'train_samples_per_second': 4.817, 'train_steps_per_second': 2.89, 'total_flos': 845886259200.0, 'train_loss': 9.960376993815105, 'epoch': 10.0})

In [ ]:
def generate_instruction(prompt):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

#  Try it out
test_prompt = "Create a site for a restaurant"
print("Prompt:", test_prompt)
print("Instruction:", generate_instruction(test_prompt))

Prompt: Create a site for a restaurant
Instruction: Créer a Créer a site pour un restaurant Créer a a site pour un restaurant


In [ ]:
#  Try it out
test_prompt = "build super market website"
print("Prompt:", test_prompt)
print("Instruction:", generate_instruction(test_prompt))

Prompt: build super market website
Instruction: build super super market


In [ ]:
#  Try it out
test_prompt = "Make a blog for travel stories"
print("Prompt:", test_prompt)
print("Instruction:", generate_instruction(test_prompt))

Prompt: Make a blog for travel stories
Instruction: Make a blog for travel travel stories travel.


**Prompt → Instruction Converter (Using Hugging Face Dataset)**

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import torch

In [ ]:
dataset = load_dataset("anon8231489123/text2web", split="train")

DatasetNotFoundError: Dataset 'anon8231489123/text2web' doesn't exist on the Hub or cannot be accessed.